In [ ]:
import pandas as pd

In [ ]:
df_model_before_Pettit=pd.read_csv("df_model_before_Pettit.csv")
df_model_after_Pettit=pd.read_csv("df_model_after_Pettit.csv")

In [ ]:
df_model_after_Pettit.drop(["Day_Night","Wind_Direction"],axis=1,inplace=True)


In [ ]:
from sklearn.model_selection import train_test_split


X_after = df_model_after_Pettit.drop('Temp - °C', axis=1)
y_after = df_model_after_Pettit['Temp - °C']

split = int(len(X_after) * 0.8)
X_train_after, X_test_after = X_after[:split], X_after[split:]
y_train_after, y_test_after = y_after[:split], y_after[split:]

In [ ]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

# --- Scale Features and Target ---
scaler_X = MinMaxScaler(feature_range=(-1,1))
scaler_y = MinMaxScaler(feature_range=(-1,1))

X_train_after_scaled = scaler_X.fit_transform(X_train_after)
X_test_after_scaled  = scaler_X.transform(X_test_after)
y_train_after_scaled = scaler_y.fit_transform(y_train_after.values.reshape(-1,1))
y_test_after_scaled  = scaler_y.transform(y_test_after.values.reshape(-1,1))

In [ ]:
def create_sequences(X, y, time_steps=60):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        Xs.append(X[i:(i + time_steps)])  # Fixed indexing
        ys.append(y[i + time_steps])
    return np.array(Xs), np.array(ys)


# --- After Pettit ---
X_train_seq_after, y_train_seq_after   = create_sequences(X_train_after_scaled, y_train_after_scaled,time_steps=60)
X_test_seq_after, y_test_seq_after     = create_sequences(X_test_after_scaled, y_test_after_scaled, time_steps=60)

print("After Pettit train:", X_train_seq_after.shape, y_train_seq_after.shape)

After Pettit train: (26532, 60, 8) (26532, 1)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

def build_lstm(input_shape):
  model = Sequential()
  model.add(LSTM(64, activation='tanh', return_sequences=True, input_shape=(X_train_seq_after.shape[1], X_train_seq_after.shape[2])))
  model.add(Dropout(0.2))
  model.add(LSTM(32))
  model.add(Dense(1))
  model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
  return model

model_after  = build_lstm((X_train_seq_after.shape[1], X_train_seq_after.shape[2]))

/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
from tensorflow.keras import callbacks

reduce_lr = callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6
)

In [ ]:

# --- After Pettit ---
history_after = model_after.fit(
    X_train_seq_after, y_train_seq_after,
    epochs=50, batch_size=32,
    validation_data=(X_test_seq_after, y_test_seq_after),
    callbacks=[reduce_lr],
    verbose=1
)

Epoch 1/50
831/831 ━━━━━━━━━━━━━━━━━━━━ 31s 33ms/step - loss: 0.0270 - val_loss: 0.0073 - learning_rate: 0.0010
Epoch 2/50
831/831 ━━━━━━━━━━━━━━━━━━━━ 40s 32ms/step - loss: 0.0070 - val_loss: 0.0044 - learning_rate: 0.0010
Epoch 3/50
831/831 ━━━━━━━━━━━━━━━━━━━━ 41s 32ms/step - loss: 0.0048 - val_loss: 0.0043 - learning_rate: 0.0010
Epoch 4/50
831/831 ━━━━━━━━━━━━━━━━━━━━ 40s 31ms/step - loss: 0.0042 - val_loss: 0.0037 - learning_rate: 0.0010
Epoch 5/50
831/831 ━━━━━━━━━━━━━━━━━━━━ 42s 32ms/step - loss: 0.0036 - val_loss: 0.0034 - learning_rate: 0.0010
Epoch 6/50
831/831 ━━━━━━━━━━━━━━━━━━━━ 40s 31ms/step - loss: 0.0034 - val_loss: 0.0034 - learning_rate: 0.0010
Epoch 7/50
831/831 ━━━━━━━━━━━━━━━━━━━━ 41s 31ms/step - loss: 0.0033 - val_loss: 0.0028 - learning_rate: 0.0010
Epoch 8/50
831/831 ━━━━━━━━━━━━━━━━━━━━ 42s 32ms/step - loss: 0.0031 - val_loss: 0.0035 - learning_rate: 0.0010
Epoch 9/50
831/831 ━━━━━━━━━━━━━━━━━━━━ 26s 31ms/step - loss: 0.0031 - val_loss: 0.0028 - learning_rate:

In [ ]:

y_pred_after = model_after.predict(X_test_seq_after)
y_pred_after_actual = scaler_y.inverse_transform(y_pred_after)
y_test_after_actual  = scaler_y.inverse_transform(y_test_seq_after)

207/207 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step


In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np


mae_after = mean_absolute_error(y_test_after_actual, y_pred_after_actual)
mse_after = mean_squared_error(y_test_after_actual, y_pred_after_actual)
rmse_after = np.sqrt(mse_after)
r2_after = r2_score(y_test_after_actual, y_pred_after_actual)

print("=== After Pettitt ===")
print(f"MAE  : {mae_after:.4f}")
print(f"MSE  : {mse_after:.4f}")
print(f"RMSE : {rmse_after:.4f}")
print(f"R²   : {r2_after:.4f}")

=== After Pettitt ===
MAE  : 0.4163
MSE  : 0.3591
RMSE : 0.5992
R²   : 0.9688


In [ ]:
for i in range(10):
    actual = y_test_after_actual[i][0]  # Extract scalar
    pred = y_pred_after_actual[i][0]    # Extract scalar
    print(f"Actual: {actual:.2f}, Predicted: {pred:.2f}")


Actual: 25.30, Predicted: 26.50
Actual: 19.90, Predicted: 26.51
Actual: 25.60, Predicted: 26.52
Actual: 25.40, Predicted: 26.52
Actual: 24.00, Predicted: 26.52
Actual: 22.80, Predicted: 26.52
Actual: 32.10, Predicted: 26.51
Actual: 35.50, Predicted: 26.51
Actual: 25.10, Predicted: 26.51
Actual: 28.90, Predicted: 26.51
